In [ ]:
import glob

import dask
import dask.array as da
import imageio
import numpy as np

from aicsimageio.writers import OmeZarrWriter
from aicsimageio.types import PhysicalPixelSizes
from skimage import transform

In [ ]:
filenames = sorted(glob.glob("data/E100/E100_data_tiffstack/*.tif"))
sample = imageio.imread(filenames[0])

In [ ]:
lazy_arrays = [dask.delayed(imageio.imread)(fn) for fn in filenames]
lazy_arrays = [da.from_delayed(x, shape=sample.shape, dtype=sample.dtype)
               for x in lazy_arrays]
data = da.stack(lazy_arrays)
data = data.rechunk(chunks=(49,421,421))

def resample(arr):
    return transform.resize(arr, (100, 200, 200), preserve_range=True, anti_aliasing=True).astype(arr.dtype)

data = data.map_blocks(resample, chunks=(100, 200, 200))

In [ ]:
%%time

with dask.config.set(scheduler='single-threaded'):
    # Write out in chunks for data analysis
    data.to_zarr("data/E100/E100_data", overwrite=True)
    
    # Write out in viewer-ready multiscale image pyramid
    writer = OmeZarrWriter("data/E100/E100_view")
    writer.write_image(
        data[:256,:1024,:1024], 
        "data",
        physical_pixel_sizes=PhysicalPixelSizes(0.25,0.10,0.10), 
        channel_names=None, 
        channel_colors=None,
        scale_num_levels=4, 
        scale_factor=2,
        dimension_order='ZYX',
    )    
    
    